In [1]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
78,application_1608823814115_0001,pyspark,idle,Link,Link


SparkSession available as 'spark'.

In [2]:
import hashlib
from datetime import datetime
from graphframes import *
from pyspark.sql import functions as func
from pyspark.sql.types import FloatType
from pyspark.sql.window import Window
import hsfs
from hops import hdfs
import os
from pyspark.sql import SQLContext

In [3]:
def hashnode(x):
    return hashlib.sha1(x.encode("UTF-8")).hexdigest()[:8]

hashnode_udf = func.udf(hashnode)

In [4]:
# Create a connection
connection = hsfs.connection()
# Get the feature store handle for the project's feature store
fs = connection.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

In [5]:
edge_fg = fs.get_feature_group('transactions_fg', 1)
node_fg = fs.get_feature_group('account_features', 1)

In [6]:
node_fg.show(5)

+---------------+--------------+-------+---------+
|initial_deposit|tx_behavior_id|acct_id|prior_sar|
+---------------+--------------+-------+---------+
|       84442.19|             1|      0|        0|
|       75795.44|             1|      1|        0|
|       42057.16|             1|      2|        0|
|       25891.68|             1|      3|        0|
|       51127.47|             1|      4|        0|
+---------------+--------------+-------+---------+
only showing top 5 rows

In [7]:
edge_fg.show(5)

+------+--------+--------------+------+--------+------+-------+-------+
|is_sar|alert_id|tran_timestamp|target|base_amt|source|tran_id|tx_type|
+------+--------+--------------+------+--------+------+-------+-------+
|     0|      -1|   1.4832288E9|  3259| 9405.71|  1767|      1|      4|
|     0|      -1|   1.4832288E9|  5141| 6884.54|  7363|      2|      4|
|     0|      -1|   1.4832288E9|  9532|  7968.4|  7585|      3|      4|
|     0|      -1|   1.4832288E9|  8792| 9042.67|  1750|      4|      4|
|     0|      -1|   1.4832288E9|  4670| 4692.79|  9060|      5|      4|
+------+--------+--------------+------+--------+------+-------+-------+
only showing top 5 rows

In [8]:
edge_df = edge_fg.read()

In [9]:
edge_df.count()

1029696

In [10]:
edge_df.where(func.col("tran_timestamp")==1.483488E9).count()

1465

In [11]:
edge_df.orderBy('tran_id').show()

+------+--------+--------------+------+--------+------+-------+-------+
|is_sar|alert_id|tran_timestamp|target|base_amt|source|tran_id|tx_type|
+------+--------+--------------+------+--------+------+-------+-------+
|     0|      -1|   1.4832288E9|  3259| 9405.71|  1767|      1|      4|
|     0|      -1|   1.4832288E9|  5141| 6884.54|  7363|      2|      4|
|     0|      -1|   1.4832288E9|  9532|  7968.4|  7585|      3|      4|
|     0|      -1|   1.4832288E9|  8792| 9042.67|  1750|      4|      4|
|     0|      -1|   1.4832288E9|  4670| 4692.79|  9060|      5|      4|
|     0|      -1|   1.4832288E9|  3861| 4089.65|  8752|      6|      4|
|     0|      -1|   1.4832288E9|  3805| 3055.04|  9645|      7|      4|
|     0|      -1|   1.4832288E9|  9280| 6473.45|  7150|      8|      4|
|     0|      -1|   1.4832288E9|  6369| 2583.42|  1685|      9|      4|
|     0|      -1|   1.4832288E9|  8255| 6753.04|  7846|     10|      4|
|     0|      -1|   1.4832288E9|  5957| 1743.57|   878|     11| 

In [87]:
tumbling_wind_tr = edge_df.groupBy(func.window(func.col("tran_timestamp").cast("timestamp"), "4 week"))\
  .agg(func.collect_list("tran_id").alias("tran_id_list"))\
  .select("window.start", "window.end", func.explode("tran_id_list").alias("tran_id"))\
  .orderBy("tran_id")

In [88]:
tumbling_wind_tr.show()

+-------------------+-------------------+-------+
|       window.start|         window.end|tran_id|
+-------------------+-------------------+-------+
|2016-12-29 00:00:00|2017-01-26 00:00:00|      1|
|2016-12-29 00:00:00|2017-01-26 00:00:00|      2|
|2016-12-29 00:00:00|2017-01-26 00:00:00|      3|
|2016-12-29 00:00:00|2017-01-26 00:00:00|      4|
|2016-12-29 00:00:00|2017-01-26 00:00:00|      5|
|2016-12-29 00:00:00|2017-01-26 00:00:00|      6|
|2016-12-29 00:00:00|2017-01-26 00:00:00|      7|
|2016-12-29 00:00:00|2017-01-26 00:00:00|      8|
|2016-12-29 00:00:00|2017-01-26 00:00:00|      9|
|2016-12-29 00:00:00|2017-01-26 00:00:00|     10|
|2016-12-29 00:00:00|2017-01-26 00:00:00|     11|
|2016-12-29 00:00:00|2017-01-26 00:00:00|     12|
|2016-12-29 00:00:00|2017-01-26 00:00:00|     13|
|2016-12-29 00:00:00|2017-01-26 00:00:00|     14|
|2016-12-29 00:00:00|2017-01-26 00:00:00|     15|
|2016-12-29 00:00:00|2017-01-26 00:00:00|     16|
|2016-12-29 00:00:00|2017-01-26 00:00:00|     17|


In [89]:
edge_fg_with_tumbling_wind = edge_df.join(tumbling_wind_tr,["tran_id"])

In [90]:
edge_fg_with_tumbling_wind.show()

+-------+------+--------+--------------+------+--------+------+-------+-------------------+-------------------+
|tran_id|is_sar|alert_id|tran_timestamp|target|base_amt|source|tx_type|       window.start|         window.end|
+-------+------+--------+--------------+------+--------+------+-------+-------------------+-------------------+
|      1|     0|      -1|   1.4832288E9|  3259| 9405.71|  1767|      4|2016-12-29 00:00:00|2017-01-26 00:00:00|
|      2|     0|      -1|   1.4832288E9|  5141| 6884.54|  7363|      4|2016-12-29 00:00:00|2017-01-26 00:00:00|
|      3|     0|      -1|   1.4832288E9|  9532|  7968.4|  7585|      4|2016-12-29 00:00:00|2017-01-26 00:00:00|
|      4|     0|      -1|   1.4832288E9|  8792| 9042.67|  1750|      4|2016-12-29 00:00:00|2017-01-26 00:00:00|
|      5|     0|      -1|   1.4832288E9|  4670| 4692.79|  9060|      4|2016-12-29 00:00:00|2017-01-26 00:00:00|
|      6|     0|      -1|   1.4832288E9|  3861| 4089.65|  8752|      4|2016-12-29 00:00:00|2017-01-26 00

In [91]:
edge_fg_with_tumbling_wind = edge_fg_with_tumbling_wind.toDF("tran_id","is_sar","alert_id","tran_timestamp","target","base_amt","source","tx_type","window_start","window_end")

In [92]:
edge_fg_with_tumbling_wind = edge_fg_with_tumbling_wind.where(func.col("window_start")=="2016-12-29 00:00:00")

In [93]:
only_sar_edge_df = edge_fg_with_tumbling_wind.filter(func.col('alert_id') != -1)
only_normal_edge_df =  edge_fg_with_tumbling_wind.filter(func.col('alert_id') == -1)

In [94]:
only_normal_edge_df.show(5)

+-------+------+--------+--------------+------+--------+------+-------+-------------------+-------------------+
|tran_id|is_sar|alert_id|tran_timestamp|target|base_amt|source|tx_type|       window_start|         window_end|
+-------+------+--------+--------------+------+--------+------+-------+-------------------+-------------------+
|      1|     0|      -1|   1.4832288E9|  3259| 9405.71|  1767|      4|2016-12-29 00:00:00|2017-01-26 00:00:00|
|      2|     0|      -1|   1.4832288E9|  5141| 6884.54|  7363|      4|2016-12-29 00:00:00|2017-01-26 00:00:00|
|      3|     0|      -1|   1.4832288E9|  9532|  7968.4|  7585|      4|2016-12-29 00:00:00|2017-01-26 00:00:00|
|      4|     0|      -1|   1.4832288E9|  8792| 9042.67|  1750|      4|2016-12-29 00:00:00|2017-01-26 00:00:00|
|      5|     0|      -1|   1.4832288E9|  4670| 4692.79|  9060|      4|2016-12-29 00:00:00|2017-01-26 00:00:00|
+-------+------+--------+--------------+------+--------+------+-------+-------------------+-------------

In [95]:
only_sar_edge_df.show(5)

+-------+------+--------+--------------+------+--------+------+-------+-------------------+-------------------+
|tran_id|is_sar|alert_id|tran_timestamp|target|base_amt|source|tx_type|       window_start|         window_end|
+-------+------+--------+--------------+------+--------+------+-------+-------------------+-------------------+
|     98|     1|      16|   1.4832288E9|  5688|  108.62|  2298|      4|2016-12-29 00:00:00|2017-01-26 00:00:00|
|    108|     1|      26|   1.4832288E9|  9601|  183.25|  8627|      4|2016-12-29 00:00:00|2017-01-26 00:00:00|
|    135|     1|      15|   1.4832288E9|  8359|  142.71|  2756|      4|2016-12-29 00:00:00|2017-01-26 00:00:00|
|    137|     1|       9|   1.4832288E9|  7702|  132.47|  7605|      4|2016-12-29 00:00:00|2017-01-26 00:00:00|
|    218|     1|      17|   1.4832288E9|  7377|  119.51|  5891|      4|2016-12-29 00:00:00|2017-01-26 00:00:00|
+-------+------+--------+--------------+------+--------+------+-------+-------------------+-------------

In [96]:
only_sar_edge_df.count()

79

In [97]:
sar_sources = only_sar_edge_df.select("source")
sar_targets = only_sar_edge_df.select("target")
sar_nodes = sar_sources.union(sar_targets).toDF("id").dropDuplicates()
sar_nodes.count()

sar_edges = only_sar_edge_df.select("source", "target").toDF("src", "dst")
sar_edges.count()

79

In [98]:
# Now lets construct the graph
g_sar = GraphFrame(sar_nodes,sar_edges)
sc.setCheckpointDir("hdfs:///Projects/{}/Logs/sc".format(hdfs.project_name()))
cc_sar = g_sar.connectedComponents()

In [99]:
cc_sar.cache().show()

+----+---------+
|  id|component|
+----+---------+
|1127|     1127|
|9403|     1794|
|6176|     1607|
|2238|     2227|
|8627|     8627|
|6378|      635|
|8628|      635|
|2739|     2491|
|8442|     1794|
|7188|     1127|
|6095|      123|
|9270|     2227|
|4919|     4616|
|3727|     3727|
|5891|     5126|
|9601|     8627|
|3792|     1964|
|2227|     2227|
|5787|     1607|
|5084|     3738|
+----+---------+
only showing top 20 rows

In [100]:
cc_sar.groupBy('component').count().select('count').dropDuplicates().orderBy('count').show()

+-----+
|count|
+-----+
|    2|
|    3|
|    4|
|    5|
|    6|
|    7|
+-----+

In [115]:
cc_sar_grouped = cc_sar.groupBy('component').count().where(func.col('count')>=7).drop('count')
cc_sar = cc_sar.join(cc_sar_grouped,['component'])
cc_sar.show()

+---------+----+
|component|  id|
+---------+----+
|     2227|2238|
|     2227|9270|
|     2227|2227|
|     2227|7709|
|     2227|2429|
|     2227|4431|
|     2227|8397|
+---------+----+

In [116]:
sar_cc_grouped = cc_sar.join(
    only_sar_edge_df,
    [(only_sar_edge_df.source==cc_sar.id)|(only_sar_edge_df.target==cc_sar.id)],
    how="left"
).dropDuplicates(subset=['tran_id'])
sar_cc_grouped.show()

+---------+----+---------+----+-------+------+--------+--------------+------+--------+------+-------+-------------------+-------------------+
|component|  id|component|  id|tran_id|is_sar|alert_id|tran_timestamp|target|base_amt|source|tx_type|       window_start|         window_end|
+---------+----+---------+----+-------+------+--------+--------------+------+--------+------+-------+-------------------+-------------------+
|     2227|7709|     2227|7709|  13830|     1|      53|   1.4840064E9|  7709|  147.41|  4431|      4|2016-12-29 00:00:00|2017-01-26 00:00:00|
|     2227|9270|     2227|9270|  14488|     1|      53|   1.4840928E9|  9270|  147.41|  4431|      4|2016-12-29 00:00:00|2017-01-26 00:00:00|
|     2227|2227|     2227|2227|  28162|     1|      53|   1.4848704E9|  2227|  147.41|  4431|      4|2016-12-29 00:00:00|2017-01-26 00:00:00|
|     2227|2238|     2227|2238|  18229|     1|      53|   1.4842656E9|  2238|  147.41|  4431|      4|2016-12-29 00:00:00|2017-01-26 00:00:00|
|     

In [117]:
sar_cc_grouped.count()

6

In [118]:
only_sar_edge_df.count()

25

In [119]:
only_sar_edge_df = sar_cc_grouped

In [120]:
only_sar_edge_df.show(5)

+---------+----+---------+----+-------+------+--------+--------------+------+--------+------+-------+-------------------+-------------------+
|component|  id|component|  id|tran_id|is_sar|alert_id|tran_timestamp|target|base_amt|source|tx_type|       window_start|         window_end|
+---------+----+---------+----+-------+------+--------+--------------+------+--------+------+-------+-------------------+-------------------+
|     2227|7709|     2227|7709|  13830|     1|      53|   1.4840064E9|  7709|  147.41|  4431|      4|2016-12-29 00:00:00|2017-01-26 00:00:00|
|     2227|9270|     2227|9270|  14488|     1|      53|   1.4840928E9|  9270|  147.41|  4431|      4|2016-12-29 00:00:00|2017-01-26 00:00:00|
|     2227|2227|     2227|2227|  28162|     1|      53|   1.4848704E9|  2227|  147.41|  4431|      4|2016-12-29 00:00:00|2017-01-26 00:00:00|
|     2227|2238|     2227|2238|  18229|     1|      53|   1.4842656E9|  2238|  147.41|  4431|      4|2016-12-29 00:00:00|2017-01-26 00:00:00|
|     

In [124]:
#only_sar_edge_df.orderBy(func.col("component")).show(10)

In [126]:
only_sar_edge_df.count()

6

In [36]:
only_normal_edge_df.count()

1028964

In [37]:
only_sar_edge_df_grouped = only_sar_edge_df.groupBy('component').agg(func.min("tran_timestamp"),func.max("tran_timestamp")).toDF("component", "window_start", "window_end")
only_sar_edge_df_grouped.show(5)

+---------+------------+-----------+
|component|window_start| window_end|
+---------+------------+-----------+
|     1127| 1.4833152E9|1.4850432E9|
|     1152|  1.516752E9|1.5170976E9|
|      635|  1.483488E9|1.4957568E9|
|      399| 1.5412032E9|1.5428448E9|
|     3396| 1.4832288E9|1.5060384E9|
+---------+------------+-----------+
only showing top 5 rows

In [38]:
only_sar_edges_df_windows = only_sar_edge_df.join(only_sar_edge_df_grouped,["component"])

In [39]:
only_sar_edges_df_windows.show()

+---------+----+------+-------+--------------+--------+------+--------+------+-------+------------+-----------+
|component|  id|is_sar|tran_id|tran_timestamp|base_amt|target|alert_id|source|tx_type|window_start| window_end|
+---------+----+------+-------+--------------+--------+------+--------+------+-------+------------+-----------+
|     1127|3320|     1|   2135|   1.4833152E9|  170.32|  8279|      11|  3320|      4| 1.4833152E9|1.4850432E9|
|     1127|8279|     1|  30182|   1.4850432E9|  170.32|  8279|      11|  9173|      4| 1.4833152E9|1.4850432E9|
|     1127|1127|     1|  16629|   1.4841792E9|  170.32|  8279|      11|  1127|      4| 1.4833152E9|1.4850432E9|
|     1127|7188|     1|   9861|   1.4837472E9|  170.32|  8279|      11|  7188|      4| 1.4833152E9|1.4850432E9|
|     1127|8279|     1|  24020|   1.4846112E9|  170.32|  8279|      11|  4902|      4| 1.4833152E9|1.4850432E9|
|     1152|7824|     1| 559198|   1.5169248E9|   82.04|  7824|      65|  8117|      4|  1.516752E9|1.517

In [40]:
only_sar_edges_df_windows.count()

732

In [42]:
only_sar_edges_df_windows.where((only_sar_edges_df_windows.window_start == 1.4833152E9) & (only_sar_edges_df_windows.window_end == 1.4850432E9)).show()

+---------+----+------+-------+--------------+--------+------+--------+------+-------+------------+-----------+
|component|  id|is_sar|tran_id|tran_timestamp|base_amt|target|alert_id|source|tx_type|window_start| window_end|
+---------+----+------+-------+--------------+--------+------+--------+------+-------+------------+-----------+
|     1127|8279|     1|  16629|   1.4841792E9|  170.32|  8279|      11|  1127|      4| 1.4833152E9|1.4850432E9|
|     1127|8279|     1|  30182|   1.4850432E9|  170.32|  8279|      11|  9173|      4| 1.4833152E9|1.4850432E9|
|     1127|8279|     1|   2135|   1.4833152E9|  170.32|  8279|      11|  3320|      4| 1.4833152E9|1.4850432E9|
|     1127|4902|     1|  24020|   1.4846112E9|  170.32|  8279|      11|  4902|      4| 1.4833152E9|1.4850432E9|
|     1127|7188|     1|   9861|   1.4837472E9|  170.32|  8279|      11|  7188|      4| 1.4833152E9|1.4850432E9|
+---------+----+------+-------+--------------+--------+------+--------+------+-------+------------+-----

In [48]:
only_normal_edges_df_windows = only_sar_edge_df_grouped.select("window_start", "window_end").join(
    only_normal_edge_df,
    [(only_normal_edge_df.tran_timestamp>=only_sar_edge_df_grouped.window_start)&(only_normal_edge_df.tran_timestamp<=only_sar_edge_df_grouped.window_end)],
    how="left"
)

In [49]:
only_normal_edges_df_windows.show()

+------------+-----------+------+-------+--------------+-------+--------+------+--------+------+
|window_start| window_end|is_sar|tran_id|tran_timestamp|tx_type|base_amt|target|alert_id|source|
+------------+-----------+------+-------+--------------+-------+--------+------+--------+------+
| 1.4833152E9|1.4850432E9|     0|   1498|   1.4833152E9|      4| 4300.78|  2341|      -1|  1712|
| 1.4833152E9|1.4850432E9|     0|   1499|   1.4833152E9|      4| 2702.02|  9721|      -1|  7976|
| 1.4833152E9|1.4850432E9|     0|   1500|   1.4833152E9|      4| 2987.06|  9759|      -1|  9567|
| 1.4833152E9|1.4850432E9|     0|   1501|   1.4833152E9|      4| 7066.07|  3219|      -1|  2664|
| 1.4833152E9|1.4850432E9|     0|   1502|   1.4833152E9|      4| 5341.72|  3459|      -1|  5908|
| 1.4833152E9|1.4850432E9|     0|   1503|   1.4833152E9|      4|  118.96|  2027|      -1|  5043|
| 1.4833152E9|1.4850432E9|     0|   1504|   1.4833152E9|      4| 4353.21|  6294|      -1|  6502|
| 1.4833152E9|1.4850432E9|    

In [50]:
only_normal_edges_df_windows.count()

26567736

In [51]:
only_normal_edges_df_windows = only_normal_edges_df_windows.withColumnRenamed("source", "origId")\
                                                           .withColumnRenamed("target", "destId")  
only_normal_edges_df_windows = only_normal_edges_df_windows.withColumn('target',hashnode_udf(func.concat(func.col('destId'),func.lit('_'),func.col('window_start'),func.lit('_'),func.col('window_end'))))\
                                                           .withColumn('source',hashnode_udf(func.concat(func.col('origId'),func.lit('_'),func.col('window_start'),func.lit('_'),func.col('window_end'))))            
only_normal_edges_df_windows.show()

+------------+-----------+------+-------+--------------+-------+--------+------+--------+------+--------+--------+
|window_start| window_end|is_sar|tran_id|tran_timestamp|tx_type|base_amt|destId|alert_id|origId|  target|  source|
+------------+-----------+------+-------+--------------+-------+--------+------+--------+------+--------+--------+
| 1.4833152E9|1.4850432E9|     0|   1498|   1.4833152E9|      4| 4300.78|  2341|      -1|  1712|e63c8751|465fa954|
| 1.4833152E9|1.4850432E9|     0|   1499|   1.4833152E9|      4| 2702.02|  9721|      -1|  7976|86bfcc1e|e7f1f1a1|
| 1.4833152E9|1.4850432E9|     0|   1500|   1.4833152E9|      4| 2987.06|  9759|      -1|  9567|4531cf75|d0ac2ea1|
| 1.4833152E9|1.4850432E9|     0|   1501|   1.4833152E9|      4| 7066.07|  3219|      -1|  2664|f4027061|2f4fa476|
| 1.4833152E9|1.4850432E9|     0|   1502|   1.4833152E9|      4| 5341.72|  3459|      -1|  5908|631e2e29|ec3cdd07|
| 1.4833152E9|1.4850432E9|     0|   1503|   1.4833152E9|      4|  118.96|  2027|

In [52]:
normal_sources = only_normal_edges_df_windows.select("source")
normal_targets = only_normal_edges_df_windows.select("target")
normal_nodes = normal_sources.union(normal_targets).toDF("id").dropDuplicates()
normal_edges = only_normal_edges_df_windows.select("source", "target").toDF("src", "dst")

In [53]:
# Now lets construct the graph
g_normal = GraphFrame(normal_nodes,normal_edges)

In [ ]:
sc.setCheckpointDir("hdfs:///Projects/{}/Logs/sc".format(hdfs.project_name()))
cc_normal = g_normal.connectedComponents().cache()
#scc_normal = g_normal.stronglyConnectedComponents(20).cache()

In [ ]:
cc_norm_comp_count = cc_normal.groupBy('component').count().where(func.col('count')>2)
cc_normal = cc_normal.join(cc_norm_comp_count,['component'])
cc_normal =  cc_normal.drop('count')
normal_cc_grouped = cc_normal.join(
    only_normal_edge_df,
    [(only_normal_edge_df.source==cc_normal.id)|(only_normal_edge_df.target==cc_normal.id)],
    how="left"
).dropDuplicates(subset=['tran_id'])

In [ ]:
normal_cc_grouped.show()

In [ ]:
1-1